In [2]:
#document_id	sentence_id	doc_start_index	doc_end_index	sentence_entity_start	sentence_entity_end	text_segment	semantic_type_id	entity_id

In [1]:
import pandas as pd

In [3]:
#df_checkpoint_mm_multi_sentences_671
df_result_outputs = pd.read_csv('df_checkpoint_sp21_test_sentences_llama_prompt_0929_v1_13b.tsv', sep='\t', header=0)
df_result_outputs.head()

,outputs
0,Create a comma-separated list of all medical t...
1,Create a comma-separated list of all medical t...
2,Create a comma-separated list of all medical t...
3,Create a comma-separated list of all medical t...
4,Create a comma-separated list of all medical t...


In [4]:
from ast import literal_eval
#df_result_outputs['outputs'] = df_result_outputs['outputs'].apply(literal_eval)

In [18]:
result_outputs = list(df_result_outputs['outputs'].values)

In [19]:
result_outputs[0:10]

['Create a comma-separated list of all medical terms in the original language without translations in the medical article below:\n"Nonylphenol diethoxylate inhibits apoptosis induced in PC12 cells Nonylphenol and short-chain nonylphenol ethoxylates such as NP2 EO are present in aquatic environment as wastewater contaminants, and their toxic effects on aquatic species have been reported. Apoptosis has been shown to be induced by serum deprivation or copper treatment. To understand the toxicity of nonylphenol diethoxylate, we investigated the effects of NP2 EO on apoptosis induced by serum deprivation and copper by using PC12 cell system."\n\nHere is the list of medical terms in the original language without translations:\n\nNonylphenol, diethoxylate, apoptosis, PC12, serum deprivation, copper, toxicity.',
 'Create a comma-separated list of all medical terms in the original language without translations in the medical article below:\n"Nonylphenol diethoxylate itself showed no toxicity an

In [24]:
def strip_llama_prompt(text):
    text = text.replace('Create a comma-separated list of all medical terms in the original language without translations in the medical article below:\n', '')
    if '"\n\n' in text:
        end_article = text.index('"\n\n')
        text = text[end_article+3:]
    else:
        text = ''
    return text

In [25]:
strip_llama_prompt(result_outputs[0])

'Here is the list of medical terms in the original language without translations:\n\nNonylphenol, diethoxylate, apoptosis, PC12, serum deprivation, copper, toxicity.'

In [27]:
idx = 0
stripped_results = []
for result in result_outputs:    
    res = strip_llama_prompt(result)
    #print(f'{idx} - {res}')
    stripped_results.append(res)
    idx+=1

In [28]:
stripped_results[0:10]

['Here is the list of medical terms in the original language without translations:\n\nNonylphenol, diethoxylate, apoptosis, PC12, serum deprivation, copper, toxicity.',
 'Here is the list of medical terms in the original language without translations:\n\n* Nonylphenol diethoxylate\n* apoptosis\n* PC12 cells\n* cytochrome c\n* cytosol\n* Bax\n* serum deprivation',
 'Here is the list of medical terms in the original language without translations:\n\n1. structure\n2. compounds\n3. phenomenon\n4. cell differentiation\n5. development\n6. apoptosis',
 'Here is the list of medical terms in the original language without translations:\n\nEZH2, ZFX, MEN1, CTNNB1, CDKIs',
 'Here is the list of medical terms in the original language without translations:\n\n* oncogenes\n* parathyroid tumour\n* malignant\n* benign\n* parathyroid neoplasm\n* EZH2\n* ZFX\n* DNA\n* PCR\n* sequenced',
 'Here is the list of medical terms in the original language without translations:\n\nEZH2, ZFX, parathyroid tumours, f

In [33]:
result_outputs = stripped_results

In [35]:
def unique_occurences(occurences):
    # intilize a null list
    unique_list = []

    # traverse for all elements
    for x in occurences:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)

    return unique_list

In [29]:
import re
def find_original_span(text, term, abbr=False):    
    found_original_text = ''
    
    search_text = text
    search_term = term
    if abbr and search_term in search_text:
        found_original_text = search_term
    else:       
        search_text = text.lower()
        search_term = term.lower()
        if search_term in search_text:
            found_original_text = search_term
        #else:
        #    found_original_text = fuzzy_ngram_match(term, text).lower()

    if found_original_text != '':
        print(found_original_text)
        occurrences_start = [m.start() for m in re.finditer(re.escape(found_original_text), search_text)]
        found_occurences = []
        for start_idx in occurrences_start:
            #start_idx = current_idx + text.lower()[current_idx:].index(found_original_text)
            end_idx = start_idx + len(found_original_text)
            found_occurences.append({
                'start_idx': start_idx,
                'end_idx': end_idx,
                'term': text[start_idx:end_idx]
            })
        return found_occurences
    else:
        return [{
            'start_idx': -1,
            'end_idx': -1,
            'term': 'no-match'
        }]

In [50]:
import json
import re
def convert_output_from_comma_list(text, outputs, abbr=False):
    outputs = str(outputs)
    if ':' in outputs[0:100]:
        outputs = outputs[outputs.index(':')+1:]
    outputs_replaced = outputs.replace('*', '')
    #.replace('\\n', ',')
    term_lines = [term_line.strip() for term_line in outputs_replaced.split('\n')]
    terms_from_lines = [term_line.split(',') for term_line in term_lines]
    terms = [term.strip() for term_line in terms_from_lines for term in term_line]
    print(f'terms: {terms}')
    original_terms = []
    for term in terms:
        if len(term.split(' ')) < 10: # longer terms are probably explanations
            if term.startswith('1. ') or term.startswith('2. ') or term.startswith('3. ') or term.startswith('4. ') or term.startswith('5. ') or term.startswith('6. ') or term.startswith('7. ') or term.startswith('8. ') or term.startswith('9. '):                
                term = term[3:]
            
            original_term = find_original_span(text, term, abbr)
            if len(original_term) > 0 and original_term[0]['start_idx'] > -1:
                original_terms.extend(original_term)
    
    #todo: handle nested entities
    return unique_occurences(original_terms)

In [57]:
df_sentences = pd.read_csv('../MedMentions-master/MedMentions-master/st21pv_test_sentences_tokens_combined.tsv', sep='\t', header=0)
df_sentences.head()

,doc_id,sentence_id,sentence,sentence_doc_start,sentence_doc_end,sentence_starts,tokens
0,25847295,0,Nonylphenol diethoxylate inhibits apoptosis in...,0,513,"[0, 66, 259, 340]","[{'text': 'Nonylphenol', 'start_char': 0, 'end..."
1,25847295,1,Nonylphenol diethoxylate itself showed no toxi...,514,1215,"[0, 96, 197, 540]","[{'text': 'Nonylphenol', 'start_char': 0, 'end..."
2,25847295,2,The difference in structure of the two compoun...,1216,1598,"[0, 104, 315]","[{'text': 'The', 'start_char': 0, 'end_char': ..."
3,26876532,0,EZH2 and ZFX oncogenes in malignant behaviour ...,0,426,"[0, 71, 221, 355]","[{'text': 'EZH2', 'start_char': 0, 'end_char':..."
4,26876532,1,"On one hand, this result raised the hypothesis...",427,1016,"[0, 218, 400, 488]","[{'text': 'On', 'start_char': 0, 'end_char': 2..."


In [58]:
texts = list(df_sentences['sentence'].values)

In [59]:
from ast import literal_eval
df_sentences['tokens'] = df_sentences['tokens'].apply(literal_eval)

In [ ]:
result_original = []

for text, outputs in zip(texts, result_outputs):    
    terms = convert_output_from_comma_list(text, outputs)
    result_original.append(terms)
    
    #combined = []
    #for output in outputs:
    #    combined += convert_output_from_comma_list(text, output)
    #result_original.append(combined)

In [52]:
len(result_original)

2691

In [53]:
df_small_results = pd.DataFrame({'outputs': result_outputs, 'original': result_original}) #, 'medical_term': result_medical_terms})
df_small_results.head(30)

,outputs,original
0,Here is the list of medical terms in the origi...,"[{'start_idx': 0, 'end_idx': 11, 'term': 'Nony..."
1,Here is the list of medical terms in the origi...,"[{'start_idx': 0, 'end_idx': 24, 'term': 'Nony..."
2,Here is the list of medical terms in the origi...,"[{'start_idx': 18, 'end_idx': 27, 'term': 'str..."
3,Here is the list of medical terms in the origi...,"[{'start_idx': 0, 'end_idx': 4, 'term': 'EZH2'..."
4,Here is the list of medical terms in the origi...,"[{'start_idx': 58, 'end_idx': 67, 'term': 'onc..."
5,Here is the list of medical terms in the origi...,"[{'start_idx': 25, 'end_idx': 29, 'term': 'EZH..."
6,Here is the list of medical terms in the origi...,"[{'start_idx': 0, 'end_idx': 7, 'term': 'Patie..."
7,Here is the list of medical terms in the origi...,"[{'start_idx': 0, 'end_idx': 11, 'term': 'Disc..."
8,Here is the list of medical terms in the origi...,"[{'start_idx': 30, 'end_idx': 33, 'term': 'PGA..."
9,Here is the list of medical terms in the origi...,"[{'start_idx': 0, 'end_idx': 11, 'term': 'Disc..."


In [54]:
df_outputs = df_small_results

In [60]:
df_subset = df_sentences#[0:1221] #.tail(600)

In [61]:
df_outputs['sentence'] = list(df_subset['sentence'].values)

In [62]:
df_outputs.head()

,outputs,original,sentence
0,Here is the list of medical terms in the origi...,"[{'start_idx': 0, 'end_idx': 11, 'term': 'Nony...",Nonylphenol diethoxylate inhibits apoptosis in...
1,Here is the list of medical terms in the origi...,"[{'start_idx': 0, 'end_idx': 24, 'term': 'Nony...",Nonylphenol diethoxylate itself showed no toxi...
2,Here is the list of medical terms in the origi...,"[{'start_idx': 18, 'end_idx': 27, 'term': 'str...",The difference in structure of the two compoun...
3,Here is the list of medical terms in the origi...,"[{'start_idx': 0, 'end_idx': 4, 'term': 'EZH2'...",EZH2 and ZFX oncogenes in malignant behaviour ...
4,Here is the list of medical terms in the origi...,"[{'start_idx': 58, 'end_idx': 67, 'term': 'onc...","On one hand, this result raised the hypothesis..."


In [63]:
df_outputs['entities'] = df_outputs['original']

In [35]:
#for index, row in df_outputs.iterrows():
#    text = row['sentence']
#    outputs = row['outputs']
#    entities = convert_output_from_comma_list(text, outputs)
#    df_outputs.at[index, 'entities'] = entities

In [ ]:
#document_id	sentence_id	doc_start_index	doc_end_index	
#sentence_entity_start	sentence_entity_end	text_segment	semantic_type_id	entity_id
#doc_id	sentence_id	sentence	sentence_doc_start	sentence_doc_end	tokens

In [64]:
df_outputs['document_id'] = list(df_subset['doc_id'].values)

In [65]:
df_outputs['sentence_id'] = list(df_subset['sentence_id'].values)

In [66]:
df_outputs['doc_start_index'] = list(df_subset['sentence_doc_start'].values)
df_outputs['doc_end_index'] = list(df_subset['sentence_doc_end'].values)

In [67]:
df_outputs.head()

,outputs,original,sentence,entities,document_id,sentence_id,doc_start_index,doc_end_index
0,Here is the list of medical terms in the origi...,"[{'start_idx': 0, 'end_idx': 11, 'term': 'Nony...",Nonylphenol diethoxylate inhibits apoptosis in...,"[{'start_idx': 0, 'end_idx': 11, 'term': 'Nony...",25847295,0,0,513
1,Here is the list of medical terms in the origi...,"[{'start_idx': 0, 'end_idx': 24, 'term': 'Nony...",Nonylphenol diethoxylate itself showed no toxi...,"[{'start_idx': 0, 'end_idx': 24, 'term': 'Nony...",25847295,1,514,1215
2,Here is the list of medical terms in the origi...,"[{'start_idx': 18, 'end_idx': 27, 'term': 'str...",The difference in structure of the two compoun...,"[{'start_idx': 18, 'end_idx': 27, 'term': 'str...",25847295,2,1216,1598
3,Here is the list of medical terms in the origi...,"[{'start_idx': 0, 'end_idx': 4, 'term': 'EZH2'...",EZH2 and ZFX oncogenes in malignant behaviour ...,"[{'start_idx': 0, 'end_idx': 4, 'term': 'EZH2'...",26876532,0,0,426
4,Here is the list of medical terms in the origi...,"[{'start_idx': 58, 'end_idx': 67, 'term': 'onc...","On one hand, this result raised the hypothesis...","[{'start_idx': 58, 'end_idx': 67, 'term': 'onc...",26876532,1,427,1016


In [ ]:
#df_outputs['original'] = df_outputs['original'].apply(literal_eval)
#df_outputs['entities'] = df_outputs['entities'].apply(literal_eval)

In [68]:
result_entities = []
for index, row in df_outputs.iterrows():
    for entity in row['entities']:
        if entity['start_idx'] > 0:
            result_entities.append({
                "document_id": row["document_id"],
                "sentence_id": row["sentence_id"],
                "doc_start_index": row["doc_start_index"],
                "doc_end_index": row["doc_end_index"],
                "sentence_entity_start": entity["start_idx"],
                "sentence_entity_end": entity["end_idx"],
                "text_segment": entity["term"]
            })

In [69]:
df_result_entities = pd.DataFrame(result_entities)
df_result_entities.head()

,document_id,sentence_id,doc_start_index,doc_end_index,sentence_entity_start,sentence_entity_end,text_segment
0,25847295,0,0,513,66,77,Nonylphenol
1,25847295,0,0,513,94,105,nonylphenol
2,25847295,0,0,513,370,381,nonylphenol
3,25847295,0,0,513,12,24,diethoxylate
4,25847295,0,0,513,382,394,diethoxylate


In [ ]:
# todo: remove overlaps

In [70]:
df_result_entities.to_csv('st21pv_test_0928v1llama13b_entities_nofuzzy.tsv', sep='\t', index=False)